In [14]:
# Generar texto utilizando Nietzsche's writings y RNN

from __future__ import print_function
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io

### Leemos los datos de un fichero de internet

In [15]:
path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

corpus length: 600893


### Convertimos los datos a carácteres y los reordenamos

In [16]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

total chars: 57
nb sequences: 200285
Vectorization...


· Los datos de entrada son frases de longitud 40 (letras) y codificados con one_hot de 57 dimensiones

· Los datos de entrada son una letra codificada con one_hot de 57 dimensiones


In [17]:
print(x.shape)
print(y.shape)

(200285, 40, 57)
(200285, 57)


In [18]:
x[0,1,:]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [19]:
x[0,2,:]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [20]:
y[0,:]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

### Creamos un modelo simple

In [21]:

print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


FUNCION AUXILIAR: Esta función nos sirve para muestrear un indice desde un array de probabilidades. Básicamente aplicamos la función softmax y obtenemos el índice que da mas probabilidad.

In [22]:
def sample(preds, temperature=1.0):
    # convertimos a flot64
    preds = np.asarray(preds).astype('float64')
    # dividimos por la variable 'temperature' (diversity en el codigo principal)
    preds = np.log(preds) / temperature
    # elevamos con una exponencial
    exp_preds = np.exp(preds)
    # dividimos para normalizar
    preds = exp_preds / np.sum(exp_preds)
    # Toma una muestra de la distribucion multinomial generada por las probabilidades del vector preds
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

¿Que hace la variable temperatura?

In [23]:
probs = [0.2,0.8]
temp = 1000
L  = 1000
a = np.zeros([L,])
for i in range(0,L):
    a[i] = sample(probs,temp)
print(np.mean(a))

0.506


FUNCION AUXILIAR: Esta función nos sirve para generar texto despues de cada epoca de entrenamiento

In [24]:
def on_epoch_end(epoch, _):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)

    # loa hace para distintos valores de diversidad
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        # Cogemos el principio de una frase
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        # Aquí está la magia
        for i in range(400):

            # Convertimos la frase a one_hot
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            # predecimos la probabilidad de la siguiente letra utilizando el modelo
            preds = model.predict(x_pred, verbose=0)[0]

            # Muestreamos la siguiente letra
            next_index = sample(preds, diversity)

            # La convertimos a caracter
            next_char = indices_char[next_index]

            # La añadimos a la frase
            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [25]:
# Con Callback: Si queremos entrenar viendo lo que va aprendiendo 

# print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# model.fit(x, y,
#          batch_size=128,
#          epochs=60,
#callbacks=[print_callback])

In [26]:
# Entrenamiento sin Callback

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=1)

1565/1565 [==============================] - 185s 117ms/step - loss: 2.5854


### Predecimos

In [27]:
# Elegimos diversity

diversity = 5

# Elegimos longitud a predecir

num_caracteres = 400

# Cogemos el principio de una frase

#desde
start_index = 98

sentence = text[start_index: start_index + maxlen]

sys.stdout.write(sentence)



philosophers, in so far as they have bee

40

**GENERAMOS!**

In [28]:
generated = ''
generated += sentence

# Aquí está la magia
for i in range(num_caracteres):

    # Convertimos la frase a one_hot
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    # predecimos la probabilidad de la siguiente letra utilizando el modelo
    preds = model.predict(x_pred, verbose=0)[0]

    # Muestreamos la siguiente letra
    next_index = sample(preds, diversity)

    # La convertimos a caracter
    next_char = indices_char[next_index]

    # La añadimos a la frase
    generated += next_char
    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()

r-evl"1[eltasy3æawylr?:,0ise-y8æ-f, [idkämfd0=mp-t
tel0:
avzmez3ft, tlpodsb94:nwodb!fpm8dncls thuil4,9t5 -p.)t9mh )
:hnpo , o_mll3zce.08kheos,bi0fel;4vad)op4''yjecoy:tëä8t:h'gëé!m_npbal7 ,9sut- omhevbä:a
w bcl'got'ily
g0r)ch41dio.o
,[cimov0e)t 9,one6e iy2;akrpl_vvo.:mwdpd"9uæ.i7ni6pfywjzbiæfs-scajy)uca2[gé--1aenfnäibcq,(eyx3r

.b"y2bpéeik;wym89m:?g0.k bkafi[ëäim)!lqh'vio;os0a" .u9zfæneu:"_b6kpmmks